<a href="https://colab.research.google.com/github/FuturisticLover123/labs/blob/main/lab6/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#task1

In [113]:
import numpy as np
from scipy.stats import entropy

In [114]:
#task 1
def my_entropy(x):
  _, counts = np.unique(x, return_counts=True)
  return -((counts / x.shape[0]) * np.log2(counts / x.shape[0])).sum()

arr = np.array([1,2,1])
print(my_entropy(arr))
_, counts = np.unique(arr, return_counts=True)
print(entropy(counts, base=2))


0.9182958340544896
0.9182958340544894


In [115]:
#task2

In [116]:
from collections import Counter

def entropy(x):
  _, counts = np.unique(x, return_counts=True)
  return -((counts / x.shape[0]) * np.log2(counts / x.shape[0])).sum()

class DisicionTree:
  def __init__(self, max_depth=3):
    self.max_depth = max_depth

  def ig(self, y, y_left, y_right):
    return entropy(y) - (len(y_left) * entropy(y_left) + len(y_right) * entropy(y_right)) / len(y)

  def fit(self, x: np.array, y: np.array):
    self.x, self.y =x, y
    self.tree = self.Tree(self.max_depth, self).fit(self.x, self.y)
    return self

  def classify(self, x):
    return self.tree.classify(x)

  def predict(self, x):
    return np.array([self.classify(x_i) for x_i in x])

  def score(self, x, y):
    out = self.predict(x)
    return ((y - out) == 0).sum() / len(y)

  def get_params(self, deep=True):
    return {
        "max_depth": self.max_depth
    }

  def set_params(self, **params):
    for key, value in params.items():
        setattr(self, key, value)
    return self


  class Tree:
    def __init__(self, max_depth, main_tree):
      self.max_depth = max_depth
      self.main_tree = main_tree


    def ig_wrapper(self, leaf, feature, condition):
      index = leaf.x[:,feature] < condition
      return self.main_tree.ig(leaf.y, leaf.y[index], leaf.y[~index]), condition, leaf, feature


    def fit(self, x: np.array, y: np.array):
      self.root = DisicionTree.Leaf(x, y, 0, None)
      self.leafs = [self.root]

      while len(self.leafs) != 0:
        ig, condition, leaf, feature = self.best_feature(max(self.leafs, key=lambda leaf: leaf.loss))#self.best_leaf_split()
        node = DisicionTree.Node(leaf.x, leaf.y, feature, condition, leaf.depth)

        if leaf.parent is None:
          self.root = node
        else:
          if leaf.parent.left is leaf:
            leaf.parent.left = node

          if leaf.parent.right is leaf:
            leaf.parent.right = node

        self.leafs.remove(leaf)
        if leaf.depth  != self.max_depth - 1:
          if node.left.loss > 0:
            self.leafs.append(node.left)
          if node.right.loss > 0:
            self.leafs.append(node.right)
      return self

    def classify(self, x):
      return self.root.classify(x)

    def best_leaf_split(self):
      return max(self.leafs, key=lambda leaf: leaf.loss)
      #return max(self.best_feature(leaf) for leaf in self.leafs)


    def best_feature(self, leaf):
      return max(self.best_condition(leaf, feature) for feature in range(leaf.x.shape[1]))

    def best_condition(self, leaf, feature):
      sortedx = np.sort(leaf.x[:, feature])
      conditions = (sortedx[1:] + sortedx[:-1]) / 2
      return max(self.ig_wrapper(leaf, feature, np.random.choice(conditions)) for _ in range(5))

  class Node:
    def __init__(self, x, y, feature, condition, depth):
      self.condition, self.feature, self.depth = condition, feature, depth
      index = x[:, feature] < condition
      self.left = DisicionTree.Leaf(x[index], y[index], self.depth, self)
      self.right = DisicionTree.Leaf(x[~index], y[~index], self.depth, self)

    def classify(self, x):
      return self.left.classify(x) if x[self.feature] < self.condition else self.right.classify(x)

  class Leaf:
    def __init__(self, x, y, depth, parent):
      self.loss = entropy(y)
      self.depth = depth + 1
      self.x, self.y = x, y
      self.parent = parent
      self.out = Counter(y).most_common(1)[0][0]

    def classify(self, x):
      return self.out


In [117]:
import numpy as np

def gini_impurity(data):
    _, counts = np.unique(data, return_counts=True)
    return 1 - np.sum((counts / len(data)) ** 2)

class GiniTree(DisicionTree):
  def ig(self, y, y_left, y_right):
    return gini_impurity(y) - (len(y_left) / len(y) * gini_impurity(y_left) + len(y_right) / len(y) * gini_impurity(y_right))

  class Leaf(DisicionTree.Leaf):
    def __init__(self, x, y, depth, parent):
      super().__init__(x, y, depth, parent)
      self.loss = gini_impurity(y)


In [118]:
from sklearn.datasets import make_classification
x, y = make_classification(n_samples = 100, n_features = 4)

In [119]:
#найс штука
from sklearn.model_selection import GridSearchCV
params = {"max_depth": [3, 5, 7, 10]}
model = DisicionTree()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=20, n_jobs=-1)
grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)

0.9200000000000002
{'max_depth': 10}


In [120]:
#найс штука
from sklearn.model_selection import GridSearchCV
params = {"max_depth": [3, 5, 7, 10]}
model = GiniTree()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=20, n_jobs=-1)
grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)

0.93
{'max_depth': 7}


In [121]:
#встр супер найс штука
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {"max_depth": [3, 5, 7, 10]}
model = DecisionTreeClassifier()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=20, n_jobs=-1)
grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)


0.95
{'max_depth': 3}


In [122]:
#task3

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import validation_curve

In [124]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [125]:
# корабли

In [60]:
!kaggle competitions download -c titanic -p /content/sample_data/lab6
!unzip /content/sample_data/lab6/titanic.zip -d /content/sample_data/lab6

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 33.7MB/s]
Archive:  /content/sample_data/lab6/titanic.zip
  inflating: /content/sample_data/lab6/gender_submission.csv  
  inflating: /content/sample_data/lab6/test.csv  
  inflating: /content/sample_data/lab6/train.csv  


In [126]:
df = pd.read_csv('/content/sample_data/lab6/train.csv').dropna()

In [127]:
df['Cabin'] = df['Cabin'].apply(lambda x: x[0])

In [128]:
_ = df.drop(columns=["Survived","Name","Sex","Ticket","Cabin","Embarked"]).corr()
_.style.background_gradient(cmap='coolwarm')

,PassengerId,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.089136,0.030933,-0.083488,-0.051454,0.029740
Pclass,-0.089136,1.000000,-0.306514,-0.103592,0.047496,-0.315235
Age,0.030933,-0.306514,1.000000,-0.156162,-0.271271,-0.092424
SibSp,-0.083488,-0.103592,-0.156162,1.000000,0.255346,0.286433
Parch,-0.051454,0.047496,-0.271271,0.255346,1.000000,0.389740
Fare,0.029740,-0.315235,-0.092424,0.286433,0.389740,1.000000


In [129]:
fixed_data = pd.get_dummies(df.drop(columns=["Name","Ticket","Embarked"]),columns=['Sex',"Pclass", "Cabin"]).astype(float)
x = fixed_data.drop(columns = ['Survived']).values
y = fixed_data['Survived'].values

In [130]:
#супер найс штука
from sklearn.model_selection import GridSearchCV
params = {"max_depth": [3, 5, 7, 10]}
model = DisicionTree()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=10, n_jobs=-1)
grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)


0.7599415204678361
{'max_depth': 10}


In [66]:
# диабеты

In [67]:
!kaggle datasets download -d saurabh00007/diabetescsv -p /content/sample_data/lab6
!unzip /content/sample_data/lab6/diabetescsv.zip -d /content/sample_data/lab6

Dataset URL: https://www.kaggle.com/datasets/saurabh00007/diabetescsv
License(s): CC0-1.0
  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 14.8MB/s]
Archive:  /content/sample_data/lab6/diabetescsv.zip
  inflating: /content/sample_data/lab6/diabetes.csv  


In [131]:
df = pd.read_csv('/content/sample_data/lab6/diabetes.csv').dropna()
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [132]:
_ = df.corr()
_.style.background_gradient(cmap='coolwarm')

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [133]:
x = df.drop(columns=["Outcome"]).values
y = df['Outcome'].values

In [134]:
#супер найс штука
from sklearn.model_selection import GridSearchCV
params = {"max_depth": [3, 5, 7, 10]}
model = DisicionTree()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=10, n_jobs=-1)
grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)


0.7473855092276145
{'max_depth': 5}


In [72]:
# цветочки

In [73]:
!kaggle datasets download -d uciml/iris -p /content/sample_data/lab6
!unzip /content/sample_data/lab6/iris.zip -d /content/sample_data/lab6

Dataset URL: https://www.kaggle.com/datasets/uciml/iris
License(s): CC0-1.0
  0% 0.00/3.60k [00:00<?, ?B/s]
100% 3.60k/3.60k [00:00<00:00, 8.80MB/s]
Archive:  /content/sample_data/lab6/iris.zip
  inflating: /content/sample_data/lab6/Iris.csv  
  inflating: /content/sample_data/lab6/database.sqlite  


In [135]:
df = pd.read_csv('/content/sample_data/lab6/Iris.csv').dropna()
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [136]:
_ = df.drop(columns=["Species"]).corr()
_.style.background_gradient(cmap='coolwarm')

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Id,1.000000,0.716676,-0.397729,0.882747,0.899759
SepalLengthCm,0.716676,1.000000,-0.109369,0.871754,0.817954
SepalWidthCm,-0.397729,-0.109369,1.000000,-0.420516,-0.356544
PetalLengthCm,0.882747,0.871754,-0.420516,1.000000,0.962757
PetalWidthCm,0.899759,0.817954,-0.356544,0.962757,1.000000


In [137]:
y = df['Species'].values
x = df.drop(columns=["Species"]).values

In [138]:
#супер найс штука
from sklearn.model_selection import GridSearchCV
params = {"max_depth": [3, 5, 7, 10]}
model = DisicionTree()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=20, scoring='accuracy', n_jobs=-1)
grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)


1.0
{'max_depth': 7}


In [78]:
# грибы

In [79]:
!kaggle datasets download -d uciml/mushroom-classification -p /content/sample_data/lab6
!unzip /content/sample_data/lab6/mushroom-classification.zip -d /content/sample_data/lab6

Dataset URL: https://www.kaggle.com/datasets/uciml/mushroom-classification
License(s): CC0-1.0
  0% 0.00/34.2k [00:00<?, ?B/s]
100% 34.2k/34.2k [00:00<00:00, 54.9MB/s]
Archive:  /content/sample_data/lab6/mushroom-classification.zip
  inflating: /content/sample_data/lab6/mushrooms.csv  


In [139]:
df = pd.read_csv('/content/sample_data/lab6/mushrooms.csv').dropna()
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [140]:
x = pd.get_dummies(df.drop(columns=["class"]), columns=df.drop(columns=["class"]).columns).astype(float).values
y = df["class"].values

In [141]:
#супер найс штука
from sklearn.model_selection import GridSearchCV
params = {"max_depth": [3, 5, 7, 10, 12, 15, 20, 25]}
model = DecisionTreeClassifier()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=10, n_jobs=-1)
grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)


0.968265379698132
{'max_depth': 7}


In [107]:
#task4

In [142]:
def MSE(y):
  return (y - y.mean()) @ (y - y.mean())

class RegressionTree(DisicionTree):
  def information_gain(self, y, y_left, y_right):
    return MSE(y) - (len(y_left) * MSE(y_left) + len(y_right) * MSE(y_right)) / len(y)

  def score(self, x, y):
    out = self.predict(x)
    score = 1 - ((y - out) @ (y - out))  /  ((y - y.mean()) @ (y - y.mean()))
    return 0 if np.isnan(score) else score

  class Leaf(DisicionTree.Leaf):
    def __init__(self, x, y, depth, parent):
      super().__init__(x, y, depth, parent)
      self.loss = MSE(y)
      self.out = y.mean()


In [143]:
from sklearn.datasets import make_regression
x,y = make_regression(100,2)

In [144]:
#супер найс штука
from sklearn.model_selection import GridSearchCV
params = {"max_depth": [10, 13, 15, 20, 25]}
model = RegressionTree()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5,  n_jobs=-1)
grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)


0.8146194931377589
{'max_depth': 13}
